In [317]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [318]:
import numpy as np

import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [319]:
# Global Parameters
R = 0.082057 # In atm*L/mol/K
N = 1.0 # In mols
gamma = 5.0/3.0 # Adiabatic index (5/3 for monoatomic gases)
pts = 100 # Number of points for each process

# Limits of parameters (volumes in L, pressures in atm)
v_min = 0.01
v_max = 20.0
p_min = 0.01
p_max = 20.0

In [320]:
def get_p_values(v,pi,j):
    '''
    This function calculates the p values for a given
    v values set and a polytropic process with index j.
    
    Inputs:
    v: 1d array of float values for volume
    pi: float value for inital state pressure
    j: float value for polytropic index
    
    Returns:
    p: 1d array of float pressure values
    '''
    
    p = pi*(v[0]/v)**j
    return p

In [321]:
def calculate_work(v,p):
    '''
    This function calculates the total Work done in the process
    described by the point arrays (v,p). This calculation
    is done by applying numerical integration based on:
    
    dW = -pdV
    
    The fact that all values in v are equidistant is exploited to
    simplify the calculation.
    
    Inputs:
    v: 1d array of float volume values for the process (in L)
    p: 1d array of float pressure values for the process (in atm)
    
    Returns:
    W: float value for the work done by the system (negative) or on the system (positive) (in J)
    
    '''
    W= -sum(p)*(v[pts-1] - v[0])/pts
    return W

In [322]:
def calculate_energy(vi,pi,vf,pf):
    '''
    This function calculates the total internal-energy change in any process
    that goes from (vi,pi) to (vf,pf). This calculation
    is done exploiting the fact that internal-energy is a function of state.
    
    dU = 3/2*N*R*dT
    
    Inputs:
    vi: float value for initial volume (in L)
    pi: float value for initial pressure (in atm)
    vf: float value for final volume (in L)
    pf: float value for final pressure (in atm)  
    
    Returns:
    dU: float value for the internal-energy change of the system (in J)
    '''
    dU = 1.5*(vf*pf - vi*pi)
    return dU

In [323]:
def update_figure(change):
    '''
    This function recalculates parameters and replots the figure
    whenever a widget value changes.
    '''
    
    vi = vi_slider.value
    pi = pi_slider.value
    vf = vf_slider.value
    j = j_slider.value
    
        
    v = np.linspace(vi,vf,pts)
    p = get_p_values(v,pi,j)
    pf = p[pts-1]
    pf_widget.value = '%.2f' % pf
    
    Ti = vi*pi/N/R
    Tf = vf*pf/N/R
    Ti_widget.value = '%.2f' % Ti
    Tf_widget.value = '%.2f' % Tf
    
    work = calculate_work(v,p)
    energy = calculate_energy(vi,pi,vf,pf)
    
    work_value_text.value = '%.2f' % work
    energy_value_text.value = '%.2f' % energy
    
    heat_value_text.value = '%.2f' % (energy - work)
    
    curve.x = v
    curve.y = p
    InitialPoint.x = [vi]
    InitialPoint.y = [pi]
    FinalPoint.x = [vf]
    FinalPoint.y = [pf]

In [324]:
def f(cahnge):
    return None

In [325]:
def change_highlighted_process(change):
    return None

In [326]:
def change_initial_point(change):
    return None

In [327]:
def change_final_volume(change):
    return None

In [328]:
# Default initial state
vi = 1.0
pi = 1.0
Ti = vi*pi/N/R
Ti_text = '%.2f' % Ti
j = 1.0 # Initial value for polytropic index

# Default final state
vf = 2.0

# Get curves
v = np.linspace(vi,vf,pts)
p = get_p_values(v,pi,j)

# Final states parameters (once p have been calculated)
pf = p[pts-1]
Tf = vf*pf/N/R
Tf_text = '%.2f' % Tf

# Energies
W = 0.0
DU = 0.0
Q = 0.0



########################
###CREATE THE FIGURES###
########################

fig_122_001 = bq.Figure(title='Prozesu sinpleak',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
                layout=widgets.Layout(width='100%')
    )


scale_x = bqs.LinearScale(min = v_min, max = v_max)
scale_y = bqs.LinearScale(min = p_min, max = p_max)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='v',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_122_001.axes = [axis_x,axis_y]


########################
####CREATE THE MARKS####
########################

marks = []

x_values = v
y_values = p

color = ['#0079c4']

curve = bqm.Lines(
                x = x_values, 
                y = y_values, 
                scales = {'x': scale_x, 'y': scale_y},
                colors = color,
                fill_colors = ['green'],
                fill_opacities = [0.35],
                fill = 'bottom',
                labels = ['Prozesua'],
                display_legend=True
)

marks.append(curve)

InitialPoint = bqm.Scatter(
    name = 'Initial Point',
    x = [vi],
    y = [pi],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['orange'],
    names = [],
    labels=['Hasierako egoera'],
    display_legend = True
)      

marks.append(InitialPoint)

FinalPoint = bqm.Scatter(
    name = 'Final Point',
    x = [vf],
    y = [pf],
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['red'],
    names = [],
    labels=['Bukaerako egoera'],
    display_legend = True
)      

marks.append(FinalPoint)

fig_122_001.marks = marks

########################
######  WIDGETS  #######
########################


## Left block (Initial state and process block)

gas_dropdown = widgets.Dropdown(
    options=[('Monoatomikoa',0), ('Diatomikoa',1)],
    value=0,
    description='Gasa',
    disabled=False,
    layout=widgets.Layout(width='100%')
)

gas_dropdown.observe(change_highlighted_process, 'value')

mol_slider = widgets.FloatSlider(
    value=1.0,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$N$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)



vi_slider = widgets.FloatSlider(
    value=1.0,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$v_i$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

vi_slider.observe(change_initial_point, 'value')


pi_slider = widgets.FloatSlider(
    value=1.0,
    min=p_min,
    max=p_max,
    step=0.1,
    description='$p_i$:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

pi_slider.observe(change_initial_point, 'value')

Ti_text = widgets.Label(value='%.2f' % Ti)
Ti_box = widgets.HBox([widgets.Label(value='$T_i$'), Ti_text, widgets.Label(value='$K$')])


process_dropdown = widgets.Dropdown(
    options=[('Isokoroa',0), ('Isobaroa',1), ('Isotermoa',2), ('Adiabatikoa',3)],
    value=0,
    description='Prozesua',
    disabled=False,
    layout=widgets.Layout(width='100%')
)

process_dropdown.observe(change_highlighted_process, 'value')


## Center block (figure block)

vf_slider = widgets.FloatSlider(
    value=2.0,
    min=v_min,
    max=v_max,
    step=0.1,
    description='$v_f$:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='100%'),
)

vf_slider.observe(change_final_volume, 'value')

pf_text = widgets.Label(value='%.2f' % pf)
pf_box = widgets.HBox([widgets.Label(value='$p_f$'), pf_text, widgets.Label(value='$atm$')])

Tf_text = widgets.Label(value='%.2f' % Tf)
Tf_box = widgets.HBox([widgets.Label(value='$T_F$'), Tf_text, widgets.Label(value='$K$')])


## Right block (energy block)

work_value_text = widgets.Label(value='%.2f' % W)
work_box = widgets.HBox([widgets.Label(value="$W:$"),work_value_text,widgets.Label(value="$J$")])


energy_value_text = widgets.Label(value='%.2f' % DU)
energy_box = widgets.HBox([widgets.Label(value="$\Delta U:$"),energy_value_text,widgets.Label(value="$J$")])

heat_value_text = widgets.Label(value='%.2f' % Q)
heat_box = widgets.HBox([widgets.Label(value="$Q:$"),heat_value_text,widgets.Label(value="$J$")])



########################
######  LAYOUT  ########
########################

## Left Block ##
left_block_122_000 = widgets.VBox([], layout=widgets.Layout(width='20%', align_items='center'))
left_block_122_000.children = [gas_dropdown, mol_slider, widgets.Label(value="Hasierako egoera:"), vi_slider, pi_slider, Ti_box,
                               process_dropdown]

## Center Block ##
center_block_122_000 = widgets.VBox([fig_122_001], layout=widgets.Layout(width='60%', align_items='center'))
center_block_122_000.children = [fig_122_001,vf_slider, pf_box, Tf_box]

## Right Block ##
right_block_122_000 = widgets.VBox([], layout=widgets.Layout(width='20%'))
right_block_122_000.children = [work_box, energy_box, heat_box]

## Main Block ##

main_block_122_000 = widgets.HBox([],layout=widgets.Layout(width='100%', align_items='center'))
main_block_122_000.children = [left_block_122_000, center_block_122_000,right_block_122_000]

main_block_122_000